# part b-1) Neural network with pytorch (basic) with auto differentiation
We are going to make neural network with 3 inputs and 2 outputs. Relu function is used for non linearity at each layer. 2 hidden layers are used -one consisting of 5 neurons and second hidden layer consisting of four neurons.We are tryint to predict more than one value.
we use dtwo non linear functions-
1. 5x1^2+x2^2+x3^2+5x1+x2+x3
2. 4x1^2+x2^2+2x3^2+4x1+x2+2x3

In [ ]:
import numpy as np
import random
from numpy import array
from numpy.random import uniform
from numpy import hstack
import torch
import torch.nn as nn

In [ ]:
#creating a class to define linear layers , here input dim represents number of input attributes, num_hidden is number of output neurons  
class Linear():
  def __init__(self,input_dim:int,num_hidden:int):
    self.weights=torch.randn(input_dim, num_hidden, requires_grad=True)
    self.bias=torch.zeros(num_hidden)

  def __call__(self, x):
    self.x = x
    output = self.x.mm(self.weights) + self.bias
    return output


  def updateWeights(self, lr):
    self.weights -= lr * self.weights.grad
      
  def gradZero(self):
    with torch.no_grad():
      # Manually zero the gradients after running the backward pass
      self.weights.grad.zero_()
  
  # def backward(self, gradient): # calculating loss function gradient
  #   self.weights_gradient = self.x.T.mm(gradient) 
  #   self.bias_gradient = gradient.sum(axis=0)
  #   self.x_gradient = gradient.mm(self.weights.T)
  #   return self.x_gradient

  # def update(self, lr): # updating weights and bias
  #   self.weights = self.weights - lr * self.weights_gradient
  #   self.bias = self.bias - lr * self.bias_gradient


In [ ]:
#defining relu function and its derative
class Relu():
  def __call__(self,input_):
    self.input_=input_
    self.output=self.input_.clamp(min=0)
   # print(input_,self.output)#torch.clamp(self.input_, min=0)#
    return self.output
  def backward(self,output_gradient):
    self.input_gradient=(self.input_>0)*output_gradient
    return self.input_gradient



In [ ]:
#creating Non linear model consisting of 3 layers-2 hidden and one output layer consisting of two output neurons.
class TorchModel():
   def __init__(self,input_dim:int,num_hidden1:int,num_hidden2:int):
    #super().__init__()
    self.linear1=Linear(input_dim, num_hidden1)
    self.relu1=Relu()
    self.linear2=Linear(num_hidden1,num_hidden2)
    self.relu2=Relu()
    self.linear3=Linear(num_hidden2,2)
  
   def __call__(self,x): # forward propagation step
    l1=self.linear1(x)
    r1=self.relu1(l1)
    l2=self.linear2(r1)
    r2=self.relu2(l2)
    l3=self.linear3(r2)
    return l3

   def updateWeights(self, lr):
    with torch.no_grad():
      self.linear1.updateWeights(lr)
      self.linear2.updateWeights(lr)
      self.linear3.updateWeights(lr)

      self.linear1.gradZero()
      self.linear2.gradZero()
      self.linear3.gradZero()



  #  def backward(self,output_gradient): # Backward Propagation step
  #   linear3_gradient=self.linear3.backward(output_gradient)
  #   relu_gradient1=self.relu2.backward(linear3_gradient)
  #   linear2_gradient=self.linear2.backward(relu_gradient1)
  #   relu_gradient2=self.relu1.backward(linear2_gradient)
  #   linear1_gradient=self.linear1.backward(relu_gradient2)
  #   return linear1_gradient
  
  #  def update(self,lr): # updating weights for each layer
  #   self.linear3.update(lr)
  #   self.linear2.update(lr)
  #   self.linear1.update(lr)




In [ ]:
#defining loss function-mse and its derivative  
class MSE:
  def __call__(self,y_pred,y_true):
    self.y_pred=y_pred
    self.y_true=y_true
    return (y_pred-y_true).pow(2).mean()
  # def backward(self):
  #  n=self.y_true.shape[0]
  #  self.gradient=2.*(self.y_pred-self.y_true)/n
  #  return self.gradient

In [ ]:
n=400  # number of input samples
d=3    # number of dimensions
num_hidden1=5 # number of neurons for first hidden layer
num_hidden2=4 # numbe rof neurons for second hidden layer
x=np.random.uniform(-1,1,(n,d)) # generating data of size=400 X 3
x_tensor=torch.tensor(x,dtype=torch.float32)

w_true=torch.tensor([[5,1,1],[4,1,1]], dtype=torch.float32).T
bias_true=torch.tensor([1], dtype=torch.float32)

y=torch.square(x_tensor).mm(w_true)+x_tensor.mm(w_true)+bias_true

print(f'x: {x.shape},weights:{w_true.shape}, bias: {bias_true.shape},y:{y.shape}')
loss=MSE() 
model=TorchModel(d,num_hidden1,num_hidden2) 
num_epochs=300
lr=0.01
for epoch in range(num_epochs):
    
    y_pred_tensor=model(x_tensor)
    loss_value=loss(y_pred_tensor,y)
    if(epoch%5==0):
      print(f'Epoch {epoch}, loss{loss_value}')
    
    loss_value.backward()
    model.updateWeights(lr)
    """
    with torch.no_grad():
      w_true -= lr * w_true.grad
      #w2_true -= lr * w2_true.grad
      # Manually zero the gradients after running the backward pass
      w_true.grad.zero_()
      #w2_true.grad.zero_()
    """
# torch_fit(x_tensor,y_true_tensor, model=model, loss=loss, lr=0.007, num_epochs=55,w1_true,w2_true) # running the non linear model



x: (400, 3),weights:torch.Size([3, 2]), bias: torch.Size([1]),y:torch.Size([400, 2])
Epoch 0, loss23.709980010986328
Epoch 5, loss18.229494094848633
Epoch 10, loss15.066071510314941
Epoch 15, loss12.844260215759277
Epoch 20, loss10.843271255493164
Epoch 25, loss8.94902515411377
Epoch 30, loss7.119389057159424
Epoch 35, loss5.434210777282715
Epoch 40, loss4.1667633056640625
Epoch 45, loss3.243192195892334
Epoch 50, loss2.6033143997192383
Epoch 55, loss2.147749662399292
Epoch 60, loss1.8174667358398438
Epoch 65, loss1.5701411962509155
Epoch 70, loss1.3811525106430054
Epoch 75, loss1.2368693351745605
Epoch 80, loss1.1237913370132446
Epoch 85, loss1.028102159500122
Epoch 90, loss0.9461012482643127
Epoch 95, loss0.8784114718437195
Epoch 100, loss0.8220407366752625
Epoch 105, loss0.7717104554176331
Epoch 110, loss0.726370096206665
Epoch 115, loss0.6849358081817627
Epoch 120, loss0.6477522850036621
Epoch 125, loss0.612626314163208
Epoch 130, loss0.5813928246498108
Epoch 135, loss0.55347883701

In [ ]:

def plot_intereactive_3d(x, y, y_pred=None):
  import plotly.graph_objects as go

  fig = go.Figure()
  fig.add_trace(go.Scatter3d(x = x[:,0],
                    y = x[:,1],
                    z = y.reshape([-1]),
                    opacity=0.5, mode='markers', name='Underlying Function'
                    ))
 
  if y_pred is not None:
    fig.add_trace(go.Scatter3d(x = x[:,0],
                   y = x[:,1],
                   z = y_pred.reshape([-1]),
                   opacity=0.5, mode='markers', name='Predicted Function'
                  ))
    
  fig.update_layout(scene = dict(
                    xaxis_title='X1',
                    yaxis_title='X2',
                    zaxis_title='Y'),
                    width=700,
                    margin=dict(r=20, b=10, l=10, t=10))
  fig.show()

In [ ]:
from sklearn.manifold import TSNE
X_red = TSNE(n_components=2).fit_transform(x_tensor)
y_true_red = TSNE(n_components=1).fit_transform(y)
y_pred_red = TSNE(n_components=1).fit_transform(y_pred.detach())
print(f'X_red: {X_red.shape}, y_true_red: {y_true_red.shape}, y_pred_red: {y_pred_red.shape}')

plot_intereactive_3d(X_red,y_true_red,y_pred_red)